This Notebook compares recommendations made by GROQ  based on several prompts.

In [24]:
import pandas as pd

Building the Training Data

In [25]:
train = pd.read_csv('../data/train.csv')

In [26]:
train.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,259,255,4,874724710,My Best Friend's Wedding (1997),Romance,4.0
1,259,286,4,874724727,"English Patient, The (1996)","Romance, War",4.0
2,259,298,4,874724754,Face/Off (1997),"Action, Sci-Fi, Thriller",4.0
3,259,185,4,874724781,Psycho (1960),"Horror, Romance, Thriller",4.0
4,259,173,4,874724843,"Princess Bride, The (1987)","Action, Adventure, Romance",4.0


The movies each user has watched in the training data

In [27]:
user_movies = train.groupby('user_id').apply(lambda x: x['movie_title'].tolist())

C:\Users\Admin\AppData\Local\Temp\ipykernel_2452\3806896237.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_movies = train.groupby('user_id').apply(lambda x: x['movie_title'].tolist())


In [28]:
""" Number of users to run the recommender on
    (Use a small sample for faster results)
    If you want to run the recommender on all users, uncomment the last line
"""
num_users = 20

# Uncomment below to run on all users
# n = user_movies.shape[0]

The most recent movies each user has watched.

In [29]:
def get_user_recent_movies(dataframe, n = 10):
    return dataframe.groupby('user_id').apply(lambda x: x['movie_title'].tail(n).to_list())

user_recent_movies = get_user_recent_movies(train)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2452\117843240.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return dataframe.groupby('user_id').apply(lambda x: x['movie_title'].tail(n).to_list())


The movies each user rated highly.

In [30]:
def get_user_top_movies(dataframe, n = 10):
    return dataframe.groupby('user_id').apply(lambda x: x.sort_values('rating', ascending=False).head(n)['movie_title'].to_list())

user_top_movies = get_user_top_movies(train)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2452\1365838253.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return dataframe.groupby('user_id').apply(lambda x: x.sort_values('rating', ascending=False).head(n)['movie_title'].to_list())


The genres of each movie.

In [31]:
movie_genres = train.groupby('movie_title').apply(lambda x: x['genres'].unique().tolist()).apply(lambda x: x[0].split(", "))

C:\Users\Admin\AppData\Local\Temp\ipykernel_2452\585781067.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  movie_genres = train.groupby('movie_title').apply(lambda x: x['genres'].unique().tolist()).apply(lambda x: x[0].split(", "))


In [32]:
user_top_movies.head()

user_id
1     [Empire Strikes Back, The (1980), Usual Suspec...
5     [Wrong Trousers, The (1993), This Is Spinal Ta...
6     [Down by Law (1986), Graduate, The (1967), Lon...
8     [Contact (1997), Die Hard (1988), Braveheart (...
10    [Amadeus (1984), Taxi Driver (1976), All About...
dtype: object

The genres for each users top rated movies

In [33]:
user_top_movie_genres = user_top_movies.apply(lambda x: pd.Series(movie_genres.loc[x].sum()).unique().tolist())

The ratings provided by each user to every movie they watched in the training data.

In [34]:
user_ratings = train.drop_duplicates(['user_id', 'movie_title']).pivot(index='user_id', columns='movie_title', values='avg_rating').fillna(0)

In [35]:
from scipy.sparse import csr_matrix
sparse_user_ratings = csr_matrix(user_ratings)

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Identifying similar users based on cosine similarity

In [37]:
def get_similar_users(sparse_user_ratings, index, n = 10):
    return pd.DataFrame(cosine_similarity(sparse_user_ratings) - np.identity(sparse_user_ratings.shape[0]), index = index, columns = index).apply(lambda x: list(x.sort_values(ascending = False).head(n).index), axis=1)

similar_users = get_similar_users(sparse_user_ratings, user_ratings.index)

In [38]:
similar_users.head()

user_id
1     [916, 457, 268, 435, 429, 823, 301, 276, 889, ...
5      [648, 407, 307, 497, 268, 276, 22, 622, 804, 70]
6       [18, 194, 716, 10, 666, 151, 321, 85, 389, 854]
8      [746, 158, 37, 352, 638, 425, 22, 627, 671, 538]
10      [6, 406, 666, 389, 321, 398, 524, 716, 194, 18]
dtype: object

Identifying Candidate movies which are the movies each user's similar users rated most highly overall.

In [39]:
def get_candidate_movies(similar_users, dataframe, n = 20):
    return similar_users.apply(lambda x: list(dataframe.loc[dataframe['user_id'].isin(x)].groupby('movie_title').sum().sort_values('rating', ascending = False).index[:n]))

candidate_movies = get_candidate_movies(similar_users, train)

In [40]:
user_prompt_data = pd.concat((user_movies, user_top_movies, user_recent_movies, similar_users, candidate_movies, user_top_movie_genres), axis = 1, keys = ['user_movies', 'user_top_movies', 'user_recent_movies', 'similar_users', 'candidate_movies', 'user_top_movie_genres'])
user_prompt_data


,user_movies,user_top_movies,user_recent_movies,similar_users,candidate_movies,user_top_movie_genres
user_id,,,,,,
1,"[Empire Strikes Back, The (1980), Monty Python...","[Empire Strikes Back, The (1980), Usual Suspec...","[Dolores Claiborne (1994), French Twist (Gazon...","[916, 457, 268, 435, 429, 823, 301, 276, 889, ...","[Chasing Amy (1997), Star Wars (1977), Empire ...","[Action, Adventure, Romance, Sci-Fi, War, Thri..."
5,"[unknown, Star Trek: First Contact (1996), Jac...","[Wrong Trousers, The (1993), This Is Spinal Ta...","[Radioland Murders (1994), Houseguest (1994), ...","[648, 407, 307, 497, 268, 276, 22, 622, 804, 70]","[Raiders of the Lost Ark (1981), Monty Python ...","[Animation, Musical, unknown, Action, Adventur..."
6,"[Kolya (1996), English Patient, The (1996), L....","[Down by Law (1986), Graduate, The (1967), Lon...","[Monty Python and the Holy Grail (1974), Bob R...","[18, 194, 716, 10, 666, 151, 321, 85, 389, 854]","[Casablanca (1942), Wizard of Oz, The (1939), ...","[unknown, Romance, Mystery, Adventure, Sci-Fi,..."
8,"[Contact (1997), Liar Liar (1997), In & Out (1...","[Contact (1997), Die Hard (1988), Braveheart (...","[Star Trek: First Contact (1996), Jurassic Par...","[746, 158, 37, 352, 638, 425, 22, 627, 671, 538]","[Blade Runner (1982), Raiders of the Lost Ark ...","[Sci-Fi, Action, Thriller, War, Adventure, unk..."
10,"[Full Monty, The (1997), L.A. Confidential (19...","[Amadeus (1984), Taxi Driver (1976), All About...","[To Wong Foo, Thanks for Everything! Julie New...","[6, 406, 666, 389, 321, 398, 524, 716, 194, 18]","[Casablanca (1942), Wizard of Oz, The (1939), ...","[Mystery, Thriller, unknown, War, Adventure, W..."
...,...,...,...,...,...,...
937,"[Contact (1997), Ulee's Gold (1997), English P...","[Boot, Das (1981), Star Wars (1977), Dead Man ...","[Ridicule (1996), Big Night (1996), Liar Liar ...","[413, 735, 136, 558, 390, 590, 63, 199, 864, 460]","[Fargo (1996), English Patient, The (1996), De...","[Action, War, Adventure, Romance, Sci-Fi, unkn..."
939,"[Jackal, The (1997), Kull the Conqueror (1997)...","[Jackal, The (1997), My Best Friend's Wedding ...","[Set It Off (1996), First Wives Club, The (199...","[718, 935, 238, 357, 181, 141, 759, 891, 15, 159]","[Independence Day (ID4) (1996), Mission: Impos...","[Action, Thriller, Romance, unknown, Adventure..."
940,"[English Patient, The (1996), Everyone Says I ...","[Titanic (1997), Air Force One (1997), Contact...","[Titanic (1997), L.A. Confidential (1997), Mot...","[404, 827, 724, 860, 149, 284, 112, 589, 740, ...","[Air Force One (1997), English Patient, The (1...","[Action, Romance, Thriller, Sci-Fi, unknown, C..."


Collaborative Filtering Prompt

In [41]:
new_line = '\n'

def collabPrompt(each, user_prompt_data, new_line = '\n'):
    return f"""I am user {each}.
The most recent ten movies I have seen are:
{", ".join(user_prompt_data.loc[each, 'user_recent_movies'])}.
My top rated movies are:
{", ".join(user_prompt_data.loc[each, 'user_top_movies'])}.
The users who are most like me are {", ".join([str(each) for each in user_prompt_data.loc[each, 'similar_users']])}.
The top movies for each of these users are:
{new_line.join([f"{each}: {', '.join(user_prompt_data.loc[each, 'user_top_movies'])}" for each in user_prompt_data.loc[each, 'similar_users']])}.
Please recommend ten movies for me to watch that I have not seen. Provide brackets around your recommendations so I can easily parse them.
For example ([Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])"""
    

In [42]:
collab_prompts = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: collabPrompt(x, user_prompt_data))

In [43]:
from dotenv import load_dotenv
from groq import Client
import os
dotenv_path = 'D:\\test\\LLM-Recommender-System-with-RAG\\key_api.env'  # Thay thế bằng đường dẫn thực tế
load_dotenv(dotenv_path)
api_key = os.getenv('GROQ_API_KEY')
client = Client(api_key=api_key)

In [44]:
collab_response = collab_prompts.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [45]:
collab_recommendations = collab_response.apply(lambda x: x.choices[0].message.content.split("[")[1].split("]")[0].split("\n"))

In [46]:
collab_recommendations.head()

user_id
1     [The Big Chill (1983), Rope (1948), The Man Wh...
5                             [The Big Lebowski (1998)]
6                                                [List]
8     [The Shawshank Redemption (1994), The Princess...
10    [The Seventh Seal (1957), 8 1/2 (1963), The Di...
Name: user_id, dtype: object

In [47]:
test = pd.read_csv('../data/test.csv')

In [48]:
user_movies_test = test.groupby('user_id').apply(lambda x: x['movie_title'].tolist())

C:\Users\Admin\AppData\Local\Temp\ipykernel_2452\134710919.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_movies_test = test.groupby('user_id').apply(lambda x: x['movie_title'].tolist())


In [49]:
collab_hits = pd.concat((user_movies, user_movies_test, collab_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'collab_recommendations']).dropna(subset = 'collab_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

Prompt Similar to Zero-Shot Paper (reduced to one prompt, added genres)

In [50]:
def genrePrompts(each, user_prompt_data, new_line = '\n'):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Their genres are: {user_prompt_data.loc[each, 'user_top_movie_genres']}.
Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?.
Please use brackets around the movies you recommend and separate the titles by new lines so I can easily parse them.
(Format Example: Here are the 10 movies recommended for you: [Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])
Answer: 
"""

In [51]:
genre_prompts = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: genrePrompts(x, user_prompt_data))

In [52]:
genre_prompts.head()

user_id
1     \nCandidate Set (candidate movies): ['Chasing ...
5     \nCandidate Set (candidate movies): ['Raiders ...
6     \nCandidate Set (candidate movies): ['Casablan...
8     \nCandidate Set (candidate movies): ['Blade Ru...
10    \nCandidate Set (candidate movies): ['Casablan...
Name: user_id, dtype: object

In [53]:
genre_response = genre_prompts.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [54]:
genre_recommendations = genre_response.apply(lambda x: x.choices[0].message.content.split("[")[-1].split("]")[0].split("\n"))

In [55]:
genre_recommendations.head()

user_id
1     [Silence of the Lambs, The (1991), Raiders of ...
5     [Monty Python and the Holy Grail (1974), Termi...
6     [Monty Python and the Holy Grail (1974), Pulp ...
8     [Blade Runner (1982), Terminator, The (1984), ...
10    [Casablanca (1942), The African Queen, The (19...
Name: user_id, dtype: object

In [56]:
genre_hits = pd.concat((user_movies, user_movies_test, genre_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'genre_recommendations']).dropna(subset = 'genre_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

Prompt Similar to Zero-Shot Paper (uses two prompts, adds genres)

In [57]:
def twoStepPrompt1(each, user_prompt_data):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Their genres are: {user_prompt_data.loc[each, 'user_top_movie_genres']}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)? 
Answer: 
"""

def twoStepPrompt2(each, user_prompt_data, response1):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Their genres are: {user_prompt_data.loc[each, 'user_top_movie_genres']}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)? 
Answer: {response1.loc[each]}.
Step 2: Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?
Please use brackets around the movies you recommend and separate the titles by new lines so I can easily parse them.
(Format Example: Here are the 10 movies recommended for you: [Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])
Answer: 
"""

In [58]:
prompt1 = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: twoStepPrompt1(x, user_prompt_data))

In [63]:
response1 = prompt1.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{'role': 'user', 'content': x}],
    max_tokens=512,
    temperature=0.7
).choices[0].message.content.strip())

# Kiểm tra kết quả
response1.head()


user_id
1     ChatCompletion(id='chatcmpl-d822f30e-b58b-4377...
5     ChatCompletion(id='chatcmpl-97fa6bff-10ab-4a40...
6     ChatCompletion(id='chatcmpl-afc7414f-3b7e-473d...
8     ChatCompletion(id='chatcmpl-a4dae5c5-71ef-47fd...
10    ChatCompletion(id='chatcmpl-c1648c89-b1db-4c6c...
Name: user_id, dtype: object

In [60]:
response1.head()

user_id
1     ChatCompletion(id='chatcmpl-bd5f6d4e-661c-4a5d...
5     ChatCompletion(id='chatcmpl-f415bfc9-073c-4ffa...
6     ChatCompletion(id='chatcmpl-5a4c9c7c-6f8b-4956...
8     ChatCompletion(id='chatcmpl-2313711a-935c-47bf...
10    ChatCompletion(id='chatcmpl-5022c932-3abe-42de...
Name: user_id, dtype: object

In [61]:
prompt2 = pd.Series(user_prompt_data.index, index = user_prompt_data.index).iloc[:num_users].apply(lambda x: twoStepPrompt2(x, user_prompt_data, response1))

In [64]:
response2 = prompt2.apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [ ]:
twoStep_recommendations = response2.apply(lambda x: x.choices[0].message.content.split("[")[-1].split("]")[0].split("\n"))

In [ ]:
twoStep_recommendations.head()

In [ ]:
twoStep_hits = pd.concat((user_movies, user_movies_test, twoStep_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'twoStep_recommendations']).dropna(subset = 'twoStep_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

In [ ]:
twoStep_hits.head()

Prompt using wikipedia movie summaries

In [ ]:
movie_wiki = pd.read_csv('../data/movie_wiki.csv')
movie_wiki.head()

In [ ]:
def wikiPrompt(each, user_prompt_data, movie_wiki, new_line = '\n'):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Summary of the movies I have watched: {new_line.join([f"{eachMovie}: {movie_wiki.loc[movie_wiki['movie_title'] == eachMovie, 'wiki_summary'].iloc[0]}" for eachMovie in user_prompt_data.loc[1, 'user_top_movies']])}
Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?.
Please use brackets around the movies you recommend and separate the titles by new lines so I can easily parse them.
(Format Example: Here are the 10 movies recommended for you: [Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])
Answer: 
"""

In [ ]:
wiki_prompts = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: wikiPrompt(x, user_prompt_data, movie_wiki))

In [ ]:
wiki_prompts.head()

In [ ]:
wiki_response = wiki_prompts.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [ ]:
wiki_recommendations = wiki_response.apply(lambda x: x.choices[0].message.content.split("[")[-1].split("]")[0].split("\n"))

In [ ]:
wiki_recommendations.head()

In [ ]:
wiki_hits = pd.concat((user_movies, user_movies_test, wiki_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'wiki_recommendations']).dropna(subset = 'wiki_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

Baseline Recommender: Recommend top 10 most popular movies to every user

In [ ]:
viewings = train.groupby('movie_title').count().sort_values('user_id', ascending=False)

In [ ]:
viewings.head()

In [ ]:
top_10_movies = viewings.head(10).index.tolist()
top_10_movies

In [ ]:
baseline_hits = pd.concat((user_movies, user_movies_test), axis=1).dropna().iloc[:num_users].apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(top_10_movies)), axis = 1)

Comparison of Prompts

In [ ]:
comparison = pd.concat((collab_hits, genre_hits, twoStep_hits, wiki_hits, baseline_hits), axis = 1, keys = ['collab_hits', 'genre_hits', 'twoStep_hits', 'wiki_hits', 'baseline_hits'])

Below values are Hit Rate in % (i.e. 100 is 100%)

In [ ]:
comparison.describe().iloc[1:] / 10 * 100